Парсер вакансий hh

Последние правки: 2022-02-06

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки на вакансии

In [ ]:
# администратор стоматологии / Медицина, фармацевтика
url_part_1 = 'https://hh.ru/search/vacancy?clusters=true&area=113&ored_clusters=true&label=not_from_agency&search_field=name&enable_snippets=true&salary=&text='

url_part_2 = '&from=suggest_post&page='

vacancy_queries = ['Администратор+стоматологической+клиники', 'Администратор+стоматологии']


Кол-во страниц, вытаскиваю из блока внизу (переход по ссылкам).

На hh пагинация страниц начинается с нуля, а номер последней страницы всегда перед кнопкой "Дальше".

Чтобы не грузить первую страницу 2 раза, сначала собираю ссылки на ней, а потом перебираю циклом оставшиеся и собираю ссылки тем же скриптом.

In [ ]:
#

links = [] # список для ссылок на страницы

for query in tqdm(vacancy_queries): # цикл для перебора запросов
    driver.get(url_part_1 + query + url_part_2 + '0')
    sleep(3)
    
    # вытаскиваю количество объявлений
    vacancy_number = (driver.find_element(By.TAG_NAME, 'div[data-qa="vacancies-search-header"]')
                   .find_element(By.TAG_NAME, 'h1')
                   .text)
    
    # считаю количество страниц
    vacancy_number = int(re.sub(r'\D', '', vacancy_number))
    page_number = math.ceil(vacancy_number / 50)
    
    # забираю ссылки с первой страницы 
    all_urls = driver.find_elements(By.TAG_NAME, 'a.bloko-link[data-qa="vacancy-serp__vacancy-title"]') # забираю ссылки

    for url_page in all_urls: # разбираю ссылки и добавляю в список
        links += [re.sub('\?from=(.)+', '', url_page.get_attribute('href'))]
        
    for i in range(1, page_number):
        driver.get(url_part_1 + query + url_part_2 + str(i))
        sleep(3)
        
        # забираю ссылки тем же кодом
        all_urls = driver.find_elements(By.TAG_NAME, 'a.bloko-link[data-qa="vacancy-serp__vacancy-title"]')
        
        for url_page in all_urls: # разбираю ссылки и добавляю в список
            links += [re.sub('\?from=(.)+', '', url_page.get_attribute('href'))]

In [ ]:
len(links)

In [ ]:
# проверяю, нет ли повторов

len(list(set(links)))

In [ ]:
links = list(set(links))

In [ ]:
links[:2]

## Код парсера

In [ ]:
data = [] # список для данных

for link in tqdm(links): # цикл для перехода по ссылкам
    
    sleep(3) # пауза перед
    driver.get(link)
    driver.implicitly_wait(5) # неявное ожидание, на случай, если страница будет долго грузиться
    
    try: # заголовок вакансии
        topic = driver.find_element(By.TAG_NAME, 'h1').text
           
    except:
        topic = 'none'
        
        
    try: # размер предлагаемой зп
        salary = driver.find_element(By.TAG_NAME, 'div[data-qa="vacancy-salary"]').text
    
    except:
        salary = 'none'
        
    
    try: # название клиники
        clinic = driver.find_element(By.TAG_NAME, 'a[data-qa="vacancy-company-name"]').text
        
    except:
        clinic = 'none'
        
    try: # адрес
        address = driver.find_element(By.TAG_NAME, 'span[data-qa=vacancy-view-raw-address]').text
        
    except:
        address = 'none'
        
    try: # текст объявления
        ads_text = driver.find_element(By.CLASS_NAME, 'g-user-content').text
                           
    except:
        ads_text = 'none'
        
    try: # Контакты. Раскрываю скрытые поля
        driver.find_element(By.CLASS_NAME, 'vacancy-contacts').find_element(By.CLASS_NAME, 'bloko-link-switch').click()
        
        try:
            address_2 = driver.find_element(By.TAG_NAME, 'span[data-qa=vacancy-view-raw-address]').text
            
        except:
            address_2 = 'none'
        
        try: # беру имя контактного лица
            contact_name = driver.find_element(By.TAG_NAME, 'p[data-qa="vacancy-contacts__fio"]').text
            
        except:
            contact_name = 'none'
            
        try: # беру e-mail
            contact_email = driver.find_element(By.TAG_NAME, 'a[data-qa="vacancy-contacts__email"]').text
            
        except:
            contact_email = 'none'
            
        try: # беру телефон
            phone = driver.find_element(By.TAG_NAME, 'p[data-qa="vacancy-contacts__phone"]').text
            
        except:
            phone = 'none'
            
    except:
        address_2 = 'none'
        contact_name = 'none'
        contact_email = 'none'
        phone = 'none'
        
        
    try: # дата создания вакансии
        ad_date = driver.find_element(By.CLASS_NAME, 'vacancy-creation-time').text
        
    except:
        ad_date = 'none'
        
        
    data.append([topic, salary, clinic, address, address_2, ads_text, contact_name, contact_email, phone, ad_date, link])

In [ ]:
# преобразовываю в датафрейм

head = ['topic', 'salary', 'clinic', 'address', 'address_2', 'ads_text', 'contact_name', 'contact_email', 'phone', 'ad_date', 'link']

work_data = pd.DataFrame(data, columns=head)

In [ ]:
work_data.shape

In [ ]:
work_data.head()

In [ ]:
current_date = datetime.datetime.now().strftime('%Y_%m_%d')

In [ ]:
# Записываю в файл

work_data.to_excel(file_path + 'hh_' + current_date + '.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.quit()